In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
# Define the data directories and image dimensions
img_width = 1280
img_height = 720
batch_size = 5

In [ ]:
# Set the path to your data directory
data_dir = "C:/Users/DELL/Desktop/datatest/D-E"

# Get the list of all folders names
image_files = os.listdir(data_dir)
print("All folders names:", image_files)

In [ ]:
# Get the list of all pass image file names
pass_dir = os.path.join(data_dir, "pass")
pass_images = [os.path.join(pass_dir, file) for file in os.listdir(pass_dir)]

# Print the lists of pass and fail image file names
print("Pass images:", pass_images)

In [ ]:
# Get the list of all fail image file names
fail_dir = os.path.join(data_dir, "fail")
fail_images = [os.path.join(fail_dir, file) for file in os.listdir(fail_dir)]

# Print the lists of pass and fail image file names
print("Fail images:", fail_images)

In [ ]:
# Split the data into train and test sets (70% train, 30% test)
pass_train, pass_test = train_test_split(pass_images, test_size=0.3, random_state=42)
fail_train, fail_test = train_test_split(fail_images, test_size=0.3, random_state=42)

# Create the train and test directories
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
os.makedirs(os.path.join(train_dir, "pass"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "fail"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "pass"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "fail"), exist_ok=True)

# Copy the pass images to the train and test directories
for file in pass_train:
    dest_file = os.path.join(train_dir, "pass", os.path.basename(file))
    if not os.path.exists(dest_file):
        shutil.copy(file, dest_file)
for file in pass_test:
    dest_file = os.path.join(test_dir, "pass", os.path.basename(file))
    if not os.path.exists(dest_file):
        shutil.copy(file, dest_file)

# Copy the fail images to the train and test directories
for file in fail_train:
    dest_file = os.path.join(train_dir, "fail", os.path.basename(file))
    if not os.path.exists(dest_file):
        shutil.copy(file, dest_file)
for file in fail_test:
    dest_file = os.path.join(test_dir, "fail", os.path.basename(file))
    if not os.path.exists(dest_file):
        shutil.copy(file, dest_file)

In [ ]:
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Flatten
model.add(Flatten())

# Fully connected layers
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Define the data generators for image augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load the training and testing data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)

In [ ]:
# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Train the model
history = model.fit(train_data, epochs=10, validation_data=test_data,callbacks=[early_stop])

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()